In [1]:
import matplotlib.pyplot as plt

import ROOT, root_pandas, root_numpy, numpy
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import numpy as np

# import necessary keras modules/functions
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Flatten, Dense, Dropout, MaxPooling2D
from keras.layers import BatchNormalization, Activation, AveragePooling2D, Concatenate

from utils import PlotLearning

Welcome to JupyROOT 6.19/01


Using TensorFlow backend.


In [2]:
# custom loss
def my_mse(true, pred):
    loss = ((true-pred)**2+abs(true-pred))/2
    return np.sum(loss)

In [3]:
file = ROOT.TFile("../FlatTree_Production/ML_Unfolding_tree_100k.root")
outTree = file.outTree
n_events = outTree.GetEntries()
n_events

54404

In [4]:
data = [list(i) for i in root_numpy.tree2array(outTree, start=0, stop = n_events)]

x = numpy.array([i[:-1] for i in data])
y = numpy.array([i[-1] for i in data])

#x = preprocessing.scale(x)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)


print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (43523, 25)
43523 train samples
10881 test samples


In [5]:
# Build the model

model = Sequential()

model.add(Dense(100, input_dim = 25, kernel_initializer='normal', activation='relu'))
model.add(Dense(80, activation='relu'))
model.add(Dense(80, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='linear'))
model.summary()
model.compile(loss=my_mse, optimizer='adam', metrics=['mse','mae','acc'])

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               2600      
_________________________________________________________________
dense_2 (Dense)              (None, 80)                8080      
_________________________________________________________________
dense_3 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_4 (Dense)              (None, 20)                1620      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 21        
Total params: 18,801
Trainable params: 18,801
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Define additional callbacks
def reduce_lr(patience_lr):
    reduce_lr_loss = keras.callbacks.ReduceLROnPlateau(monitor='val_mean_squared_error', factor=0.5, patience=patience_lr, verbose=1, min_delta=1e-4, mode='auto')
    return reduce_lr_loss

callbacks = [reduce_lr(patience_lr=20), PlotLearning(monitor=['loss','mean_squared_error'])]

In [7]:
history = model.fit(x_train, y_train, epochs=100, batch_size=64,  verbose=1, validation_split=0.2, callbacks = callbacks)

KeyboardInterrupt: 

2019-08-07 13:40:42.710418: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA


In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
y_true = y_test.flatten()
y_pred = model.predict(x_test).flatten()

In [ ]:
print(y_true, y_pred)

In [ ]:
(abs(y_true-y_pred)**2).sum()/(len(y_pred))

In [ ]:
model.metrics_names

In [ ]:
from utils import array2TH1F

In [ ]:
histo_reco = array2TH1F(numpy.array([i[-1] for i in x_test]), bins=np.array(range(0,150,1)),name="reco",title="reco")
histo_gen  = array2TH1F(y_test, bins=np.array(range(0,150,1)),name="gen",title="gen")
histo_pred = array2TH1F(model.predict(x_test), bins=np.array(range(0,150,1)),name="unfolded",title="unfolded")

In [ ]:
%jsroot on

In [ ]:
canvas = ROOT.TCanvas('canvas', 'canvas', 800, 800)
histo_gen.SetLineColor(ROOT.kRed)
histo_reco.SetLineColor(ROOT.kGreen)
histo_pred.Draw('HIST')
histo_reco.Draw('SAME')

histo_gen.Draw('SAME')
canvas.BuildLegend()
canvas.Draw()